# Quickstart

This is a quick baseline model and to help you get familiar with the data format, submission process and so on. I figure it's worth sharing for those wanting to get up and running quickly using fastai :)

This isn't a tutorial - although there are some suggestions at the bottom of the notebook for directins to explore.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Setup

Assuming provided files are present. Installing fastai (may need to restart runtime)

In [ ]:
!pip install fastai --upgrade

In [ ]:
# Extracting the files
#!unzip '/content/drive/MyDrive/indabaX_cameroon/indabaX_cameroon_image.zip'

# Load data and put in dataloaders

Fastai makes it relatively easy to get our training data from the dataframe into a format we can use for training.

In [ ]:
from fastai.vision.all import *
import pandas as pd

import numpy as np 
import matplotlib.pyplot as plt
import time
import glob
import copy
from copy import deepcopy
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
from torch.autograd import Variable
import sklearn.metrics as metrics
from torch.utils.data.sampler import SubsetRandomSampler
import datetime as dt
from os import listdir, makedirs, getcwd, remove
from mpl_toolkits.axes_grid1 import ImageGrid
import torchvision.models as models

from fastai import *
from fastai.vision import *
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import albumentations as A
import cv2


import warnings
warnings.filterwarnings('ignore')

In [ ]:
import random
from torch.utils.data import Dataset
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
import torch
torch.cuda.is_available()
torch.cuda.empty_cache()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
### for reprocducibility
def seed_all(seed): 
  torch.manual_seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [ ]:
seed=0
seed_all(seed)

In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/IndabaxCamer/Train.csv')
print(train.shape)
train.head(2)

(1548, 2)


,Image_ids,class
0,ImageID_ZWY4MO08,fruit_healthy
1,ImageID_OZQDOU6U,fruit_brownspot


In [ ]:
test = pd.read_csv('/content/gdrive/MyDrive/IndabaxCamer/Test.csv')
print(test.shape)
test.head(2)

(516, 1)


,Image_ids
0,ImageID_GXD1UROG
1,ImageID_U9W52JFZ


In [ ]:
#path = "/content/gdrive/MyDrive/IndabaxCamer/Images"

In [ ]:
seed_all(seed)
# Create the dataloaders
dls = ImageDataLoaders.from_df(train, path="/content/gdrive/MyDrive/IndabaxCamer/Images", 
                               fn_col='Image_ids', 
                               label_col='class', 
                               suff='.jpg', item_tfms=Resize(224), num_workers=0, bs=32)
dls.show_batch()

# Quick Model

No fancy training here - this only took a minute, just enough time for me to organise my previous cells a little :)

In [ ]:
seed_all(seed)
learn = cnn_learner(dls, densenet161, metrics=accuracy)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

SuggestedLRs(valley=0.0010000000474974513)

In [ ]:
# Find the good learning rate
seed_all(seed)
learn.lr_find()

In [ ]:
# Fit
seed_all(seed)
num_epochs = 6
learn.fit_one_cycle(num_epochs,  1e-3 )

epoch,train_loss,valid_loss,accuracy,time
0,0.825300,0.276107,0.893204,01:23
1,0.352573,0.136316,0.957929,01:23
2,0.174175,0.145558,0.954693,01:23
3,0.113782,0.123532,0.961165,01:23
4,0.056506,0.090430,0.970874,01:23
5,0.037816,0.092399,0.970874,01:23


In [ ]:
seed_all(seed)
learn.freeze_to(-2)
learn.fit_one_cycle(2, slice(1e-3/(2.6**4), 1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.063653,0.172543,0.967638,01:35
1,0.063727,0.239510,0.967638,01:35


In [ ]:
seed_all(seed)
learn.freeze_to(-3)
learn.fit_one_cycle(2, slice(1e-4/(2.6**4), 1e-4))

epoch,train_loss,valid_loss,accuracy,time
0,0.046081,0.159216,0.967638,01:50
1,0.025305,0.157428,0.974110,01:50


In [ ]:
seed_all(seed)
learn.unfreeze()
learn.fit_one_cycle(5, slice(5e-3/(2.6**4), 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.166490,2.350327,0.886731,01:50
1,0.237165,0.126549,0.970874,01:50
2,0.148784,0.057022,0.987055,01:49
3,0.080576,0.049473,0.993528,01:50
4,0.045495,0.048677,0.990291,01:50


In [ ]:
seed_all(seed)
learn.fit_one_cycle(6, lr_max=slice(1e-5,1e-4))

epoch,train_loss,valid_loss,accuracy,time
0,0.008587,0.053426,0.990291,01:49
1,0.016467,0.058010,0.987055,01:49
2,0.012049,0.047140,0.993528,01:49
3,0.007495,0.048678,0.993528,01:49
4,0.007157,0.048084,0.993528,01:49
5,0.007810,0.050900,0.993528,01:49


# Preds on test set

I didn't read the info page at all, so my first sub consisted of probabilities. The score of 0 was enough for me to re-evaluate and realize they want a ranked list of predictions. np.argsort to the rescue :) The way I've done it here isn't the most efficient but who cares - it runs and we can get a submission on the leaderboard asap!

EDIT: I had to come back and fix this - I missed that the row order differs between test and sample sub. 

In [ ]:
seed_all(seed)
preds = learn.get_preds(dl=dls.test_dl(test)) # Getting the predicted probabilities

In [ ]:
seed_all(seed)
# Checking the format
ss = pd.read_csv('/content/gdrive/MyDrive/IndabaxCamer/Sample_submission.csv')
ss.head(2)

,Image_ids,class
0,ImageID_GXD1UROG,NaN
1,ImageID_U9W52JFZ,NaN


In [ ]:
seed_all(seed)
preds[0][0], np.argsort(preds[0])[0] # We have probs and then index if they were sorted (low to high)

(TensorBase([4.3168e-05, 3.6324e-06, 9.9995e-01]), TensorBase([1, 0, 2]))

In [ ]:
seed_all(seed)
learn.dls.vocab # The labels in the order used by the model

['fruit_brownspot', 'fruit_healthy', 'fruit_woodiness']

In [ ]:
seed_all(seed)
# Need to make sure order is right - note staring with the highest prob
test['class'] = [learn.dls.vocab[p[2]] for p in np.argsort(preds[0])]
test.head()

,Image_ids,class
0,ImageID_GXD1UROG,fruit_woodiness
1,ImageID_U9W52JFZ,fruit_brownspot
2,ImageID_18ZU2YP5,fruit_woodiness
3,ImageID_8RBI9ZMH,fruit_brownspot
4,ImageID_ZYJELERW,fruit_woodiness


In [ ]:
seed_all(seed)
test.head()

,Image_ids,class
0,ImageID_GXD1UROG,fruit_woodiness
1,ImageID_U9W52JFZ,fruit_brownspot
2,ImageID_18ZU2YP5,fruit_woodiness
3,ImageID_8RBI9ZMH,fruit_brownspot
4,ImageID_ZYJELERW,fruit_woodiness


In [ ]:
seed_all(seed)
ss.head(2)

,Image_ids,class
0,ImageID_GXD1UROG,NaN
1,ImageID_U9W52JFZ,NaN


In [ ]:
seed_all(seed)
# Merging them into ss
ss = pd.merge(ss['Image_ids'], test[['Image_ids', 'class']], how='left', on='Image_ids')
ss.head()

,Image_ids,class
0,ImageID_GXD1UROG,fruit_woodiness
1,ImageID_U9W52JFZ,fruit_brownspot
2,ImageID_18ZU2YP5,fruit_woodiness
3,ImageID_8RBI9ZMH,fruit_brownspot
4,ImageID_ZYJELERW,fruit_woodiness


In [ ]:
seed_all(seed)
ss.to_csv('/content/gdrive/MyDrive/IndabaxCamer/s01.csv', index=False) 

In [ ]:
ss.head()

,Image_ids,class
0,ImageID_GXD1UROG,fruit_woodiness
1,ImageID_U9W52JFZ,fruit_brownspot
2,ImageID_18ZU2YP5,fruit_woodiness
3,ImageID_8RBI9ZMH,fruit_brownspot
4,ImageID_ZYJELERW,fruit_woodiness


# Where next?

This model gives me a feel for the task. But what would one do next? More training? Here are some (uninformed) suggestions:
- Explore the data.
- Everything is very...green in these pics. Perhaps we can do a color transform to bring out the features more clearly?
- ... after playing around, we can then start trying more training, bigger models etc. But don't do that until you've seen what gains you can get by better treatment of the data (cropping etc)!